<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

Classification Model | Game of Thrones</h2>

Sneha Ramu - MBAN


<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Part I: Importing Library and Reading File</h2><br>

In [1]:
# import all required libraries
import matplotlib.pyplot       as plt                   # data visualization
import numpy                   as np                    # mathematical essentials
import pandas                  as pd                    # data science essentials
import seaborn                 as sns                   # enhanced data viz
import statsmodels.formula.api as smf                   # linear modeling

from sklearn.ensemble import GradientBoostingClassifier # gbm
from sklearn.ensemble import RandomForestClassifier     # random forest

from sklearn.linear_model import LogisticRegression     # logistic regression

from sklearn.metrics import confusion_matrix            # confusion matrix
from sklearn.metrics import make_scorer                 # customizable scorer
from sklearn.metrics import roc_auc_score               # auc score

from sklearn.model_selection import RandomizedSearchCV  # hyperparameter tuning
from sklearn.model_selection import train_test_split    # train-test split

from sklearn.tree import DecisionTreeClassifier         # classification trees
from sklearn.tree import plot_tree                      # tree plots

# read file
got = pd.read_excel('./GOT_character_predictions.xlsx')

# got = pd.read_excel('./GOT_feature_rich.xlsx')

# setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

# output first 5 rows of the data
got.head(n = 5)

,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,book1_A_Game_Of_Thrones,book2_A_Clash_Of_Kings,book3_A_Storm_Of_Swords,book4_A_Feast_For_Crows,book5_A_Dance_with_Dragons,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,popularity,isAlive
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,NaN,NaN,0,0,0,0,0,1.0,0.0,0.0,NaN,0,0,NaN,11,0.605351,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,NaN,NaN,NaN,House Frey,Perra Royce,1,1,1,1,1,NaN,NaN,NaN,1.0,1,1,97.0,1,0.896321,1
2,3,Addison Hill,Ser,NaN,NaN,NaN,NaN,NaN,House Swyft,NaN,0,0,0,1,0,NaN,NaN,NaN,NaN,0,1,NaN,0,0.267559,1
3,4,Aemma Arryn,Queen,NaN,82.0,NaN,NaN,NaN,House Arryn,Viserys I Targaryen,0,0,0,0,0,NaN,NaN,NaN,0.0,1,1,23.0,0,0.183946,0
4,5,Sylva Santagar,Greenstone,Dornish,276.0,NaN,NaN,NaN,House Santagar,Eldon Estermont,0,0,0,1,0,NaN,NaN,NaN,1.0,1,1,29.0,0,0.043478,1


<h2>Declaring User - Defined Functions</h2><br>

In [2]:
# user-defined functions

#########################
# mv_flagger
#########################

def mv_flagger(df):
    """
Flags all columns that have missing values with 'm-COLUMN_NAME'.

PARAMETERS
----------
df : DataFrame to flag missing values


RETURNS
-------
DataFrame with missing value flags."""


    for col in df:

        if df[col].isnull().astype(int).sum() > 0:
            df['m_'+col] = df[col].isnull().astype(int)
            
    return df



#########################
# text_split_feature
#########################

def text_split_feature(col, df, sep=' ', new_col_name='number_of_names'):
    """
Splits values in a string Series (as part of a DataFrame) and sums the number
of resulting items. Automatically appends summed column to original DataFrame.

PARAMETERS
----------
col          : column to split
df           : DataFrame where column is located
sep          : string sequence to split by, default ' '
new_col_name : name of new column after summing split, default
               'number_of_names'
"""
    
    df[new_col_name] = 0
    
    
    for index, val in df.iterrows():
        df.loc[index, new_col_name] = len(df.loc[index, col].split(sep = ' '))

In [3]:
# record the types of names
text_split_feature(col = 'name',
                   df  = got)

got['number_of_names'].value_counts(normalize = False,
                                        sort      = False,
                                        ascending = False).sort_index()

1     591
2    1201
3      98
4       9
5      35
6      12
Name: number_of_names, dtype: int64

<h2>Part II: Dataset Anomolies</h2><br>
    
<br>a) Missing Value Detection<br>
    Explore the dataset for missing values using isnull( ) and sum( ).
    
<br>b) Develop missing value and Categorical encoding strategies <br>
    Replace missing values with categorical, discrete or continuous values accordingly. 
    

In [4]:
# check for missing values
got.isnull().sum()

S.No                             0
name                             0
title                         1008
culture                       1269
dateOfBirth                   1513
mother                        1925
father                        1920
heir                          1923
house                          427
spouse                        1670
book1_A_Game_Of_Thrones          0
book2_A_Clash_Of_Kings           0
book3_A_Storm_Of_Swords          0
book4_A_Feast_For_Crows          0
book5_A_Dance_with_Dragons       0
isAliveMother                 1925
isAliveFather                 1920
isAliveHeir                   1923
isAliveSpouse                 1670
isMarried                        0
isNoble                          0
age                           1513
numDeadRelations                 0
popularity                       0
isAlive                          0
number_of_names                  0
dtype: int64

<h3>Filling missing values of 'Title'</h3>

In [5]:
# imputing missing values of 'title'
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

got['title'].fillna(value = 'Unknown',
                         inplace = True)

got['title'].value_counts()

Unknown                                                        1008
Ser                                                             384
Maester                                                          39
Archmaester                                                      21
Lord                                                             20
Septon                                                           18
Winterfell                                                       17
Princess                                                         16
Lady                                                             12
Septa                                                            10
King in the North                                                 9
Prince                                                            9
Lord Commander of the Night's Watch                               9
Grand Maester                                                     8
Cupbearer                                       

In [6]:
# storing values
got['title_unknown'] = 0
got['title_ser']     = 0
got['title_dozen']   = 0

for index, value in got.iterrows():
    

    # Unknown title 
    if got.loc[index, 'title'] == 'Unknown':
        got.loc[index, 'title_unknown'] = 1
        
        
    # Titles with members > 100
    elif got.loc[index, 'title'] == "Ser":
        got.loc[index, 'title_ser'] = 1
        
        
    # Titles with with 10 < members < 100
    elif got.loc[index, 'title'] in ['Maester',\
                                      'Archmaester',\
                                      'Lord,Septon',\
                                      'Winterfell',\
                                      'Princess',\
                                      'Lady',\
                                      'Septa']:
        got.loc[index, 'title_dozen'] = 1

<h3>Filling missing values of 'Culture'</h3>

In [7]:
# imputing missing values of 'culture'
got['culture'].fillna(value = 'Unknown',
                         inplace = True)

got['culture'].value_counts() 

Unknown                    1269
Northmen                    124
Ironborn                    112
Free Folk                    51
Valyrian                     43
Braavosi                     42
Ghiscari                     25
Dornish                      25
Dothraki                     23
Rivermen                     19
Valemen                      19
Reach                        16
Vale mountain clans          15
Dornishmen                   14
Westeros                     12
Free folk                    11
northmen                      9
Westerman                     9
Stormlands                    7
Tyroshi                       7
Qartheen                      6
Ironmen                       5
Summer Isles                  5
Northern mountain clans       5
Lysene                        4
Westermen                     4
Crannogmen                    4
Astapori                      4
Pentoshi                      3
Meereenese                    3
First Men                     3
Lyseni  

In [8]:
# storing values
got['culture_unknown'] = 0
got['culture_hundred'] = 0
got['culture_dozen'] = 0


for index, value in got.iterrows():
    

    # Unknown Culture
    if got.loc[index, 'culture'] == 'Unknown':
        got.loc[index, 'culture_unknown'] = 1
        
        
    # Cultures with members > 100
    elif got.loc[index, 'culture'] in ['Northmen','Ironborn']:
        got.loc[index, 'culture_hundred'] = 1
        
        
    # Cultures with 10 < members < 100
    elif got.loc[index, 'culture'] in ['Free Folk',
                                        'Valyrian',
                                        'Braavos',
                                        'Ghiscari',
                                        'Dornish',
                                        'Dothraki',
                                        'Rivermen',
                                        'Valemen',
                                        'Reach',
                                        'Vale mountain clans',
                                        'Dornishme',
                                        'Westeros',
                                        'Free folk']:
        got.loc[index, 'culture_dozen'] = 1

<h3>Filling missing values of 'DOB'</h3>

In [9]:
# imputing values for 'date of birth'
bir_median = got['dateOfBirth'].median()


got['dateOfBirth'].fillna(value  = bir_median,
                          inplace = True)

got['dateOfBirth'].isnull().sum()

0

<h3>Filling missing values of 'House'</h3>

In [10]:
# imputing missing values of 'house'
got['house'].fillna(value = 'Unknown',
                         inplace = True)

got['house'].value_counts()

Unknown                                    427
Night's Watch                              105
House Frey                                  97
House Stark                                 72
House Targaryen                             62
House Lannister                             49
House Greyjoy                               41
House Tyrell                                36
House Martell                               29
House Osgrey                                21
Faith of the Seven                          17
House Arryn                                 16
House Hightower                             16
House Bracken                               14
House Botley                                13
House Bolton                                13
House Baratheon                             13
House Florent                               13
House Tully                                 12
Brave Companions                            12
House Whent                                 11
Brotherhood w

In [11]:
# storing values
got['house_unknown'] = 0
got['house_nwatch'] = 0
got['house_dozen'] = 0

for index, value in got.iterrows():
    

    # Unknown house
    if got.loc[index, 'house'] == 'Unknown':
        got.loc[index, 'house_unknown'] = 1
        
        
    # Houses with members > 100
    elif got.loc[index, 'house'] == "Night's Watch":
        got.loc[index, 'house_nwatch'] = 1
        
        
    # Houses with 10 =< members < 100
    elif got.loc[index, 'house'] in ['House Frey',
                                      'House Targaryen',
                                      'House Lannister',
                                      'House Greyjoy',
                                      'House Tyrell',
                                      'House Martell',
                                      'House Osgrey',
                                      'Faith of the Seven',
                                      'House Arryn',
                                      'House Hightower',
                                      'House Bracken',
                                      'House Botley',
                                      'House Bolton',
                                      'House Baratheon',
                                      'House Florent',
                                      'House Tully',
                                      'Brave Companions',
                                      'House Whent',
                                      'Brotherhood without banners',
                                      'House Velaryon',
                                      'House Crakehall']:
        got.loc[index, 'house_dozen'] = 1

<h3>Filling missing values of 'Mother / Father / Heir / Spouse'</h3>

In [12]:
# imputing missing values for 'mother / father / heir / spouse'
got['mother'].fillna(value  = 'Unknown',
                     inplace = True)

got['father'].fillna(value  = 'Unknown',
                     inplace = True)

got['heir'].fillna(   value = 'Unknown',
                     inplace = True)

got['spouse'].fillna( value = 'Unknown',
                     inplace = True)

<h4>Mother</h4>

In [13]:
# storing values for mother
got['mother_alive'] = 0
got['mother_dead'] = 0

for index, value in got.iterrows():

    # Mother = alive
    if got.loc[index, 'mother'] != 'Unknown' and got.loc[index, 'isAliveMother'] == 1:
        got.loc[index, 'mother_alive'] = 1
        
    # Mother = dead   
    elif got.loc[index, 'mother'] != 'Unknown' and got.loc[index, 'isAliveMother'] == 0:
        got.loc[index, 'mother_dead'] = 1

<h4>Father</h4>

In [14]:
# storing values for father
got['father_alive'] = 0
got['father_dead'] = 0

for index, value in got.iterrows():

    # Father is alive 
    if got.loc[index, 'father'] != 'Unknown' and got.loc[index, 'isAliveFather'] == 1:
        got.loc[index, 'father_alive'] = 1
        
    # Father is dead   
    elif got.loc[index, 'father'] != 'Unknown' and got.loc[index, 'isAliveFather'] == 0:
        got.loc[index, 'father_dead'] = 1

<h4>Heir</h4>

In [15]:
# storing values for heir
got['heir_alive'] = 0
got['heir_dead'] = 0

for index, value in got.iterrows():

    # Father = alive 
    if got.loc[index, 'heir'] != 'Unknown' and got.loc[index, 'isAliveHeir'] == 1:
        got.loc[index, 'heir_alive'] = 1
        
    # Father = dead   
    elif got.loc[index, 'heir'] != 'Unknown' and got.loc[index, 'isAliveHeir'] == 0:
        got.loc[index, 'heir_dead'] = 1

<h4>Spouse</h4>

In [16]:
# storing values for spouse
got['spouse_alive'] = 0
got['spouse_dead'] = 0

for index, value in got.iterrows():

    # Spouse is alive
    if got.loc[index, 'spouse'] != 'Unknown' and got.loc[index, 'isAliveSpouse'] == 1:
        got.loc[index, 'spouse_alive'] = 1
     
    # Spouse is dead
    elif got.loc[index, 'spouse'] != 'Unknown' and got.loc[index, 'isAliveSpouse'] == 0:
        got.loc[index, 'spouse_dead'] = 1

<h3>Filling missing values of 'Age'</h3>

In [17]:
#imputing missing values for age
age_median = got['age'].median()


#fill missing age with median
got['age'].fillna(value  = age_median,
                  inplace = True)

got['age'].isnull().sum()

0

<h3>Re-check for missing values</h3>

In [18]:
dictionary = ['title_unknown','title_ser','title_dozen','culture_unknown', \
              'culture_hundred' ,'culture_dozen','house_unknown','house_nwatch',\
              'house_dozen','mother_alive','mother_dead','father_alive',\
              'father_dead','heir_alive','heir_dead','spouse_alive',\
              'spouse_dead','isNoble','book1_A_Game_Of_Thrones',\
              'book2_A_Clash_Of_Kings','book3_A_Storm_Of_Swords',\
              'book4_A_Feast_For_Crows','book5_A_Dance_with_Dragons',\
              'number_of_names','dateOfBirth','age','numDeadRelations',\
              'popularity','isAlive']

In [19]:
got_df = pd.DataFrame()

got_df = got.loc[ : , dictionary ]

In [20]:
got_df.isnull().sum()

title_unknown                 0
title_ser                     0
title_dozen                   0
culture_unknown               0
culture_hundred               0
culture_dozen                 0
house_unknown                 0
house_nwatch                  0
house_dozen                   0
mother_alive                  0
mother_dead                   0
father_alive                  0
father_dead                   0
heir_alive                    0
heir_dead                     0
spouse_alive                  0
spouse_dead                   0
isNoble                       0
book1_A_Game_Of_Thrones       0
book2_A_Clash_Of_Kings        0
book3_A_Storm_Of_Swords       0
book4_A_Feast_For_Crows       0
book5_A_Dance_with_Dragons    0
number_of_names               0
dateOfBirth                   0
age                           0
numDeadRelations              0
popularity                    0
isAlive                       0
dtype: int64

<h2>Part III: Classification Model & Hyperparameter Tuning </h2><br>
    
<br>a) Declare Response variable and Explanatory variables. <br>
    
<br>b) Split the data into training and testing data. <br>

<br>c) Select variables that produce good score. <br>

<br>d) Generate a dictionary for easy targeting, from fulldata to no significant variables. <br>

<br>e) Generate default models <br>

<br>f) Generate confusion matrix <br>

<br>g) Generate hyperparameter tuning <br>

<br>h) Reset models and generate confusion matrices <br>

<h3>Obtain Response Variable Proportions for train & test set

In [21]:
# declaring explanatory variables
got_df_data = got.drop('isAlive', axis = 1)


# declaring response variable
got_df_target = got.loc[ : , 'isAlive']

In [22]:
# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            got_df_data,
            got_df_target,
            test_size    = 0.1,
            random_state = 219,
            stratify     = got_df_target) # preserving balance


# merging training data for statsmodels
got_df_train = pd.concat([x_train, y_train], axis = 1)

In [23]:
print(f"""

Response Variable Proportions (Training Set)
--------------------------------------------
{y_train.value_counts(normalize = True).round(decimals = 2)}



Response Variable Proportions (Testing Set)
--------------------------------------------
{y_test.value_counts(normalize = True).round(decimals = 2)}
""")




Response Variable Proportions (Training Set)
--------------------------------------------
1    0.75
0    0.25
Name: isAlive, dtype: float64



Response Variable Proportions (Testing Set)
--------------------------------------------
1    0.74
0    0.26
Name: isAlive, dtype: float64



<h3>Logistic Regression</h3>

<h4>Generate a default model.</h4>

In [24]:
# instantiating a logistic regression model object
logistic_full = smf.logit(formula = """isAlive ~ title_unknown +
                                                 title_ser +
                                                 title_dozen +
                                                 culture_unknown +
                                                 culture_hundred +
                                                 culture_dozen +
                                                 house_unknown +
                                                 house_nwatch +
                                                 house_dozen +
                                                 mother_alive +
                                                 mother_dead +
                                                 father_alive +
                                                 father_dead +
                                                 heir_alive +
                                                 heir_dead +
                                                 spouse_alive +
                                                 spouse_dead +
                                                 isNoble +
                                                 book1_A_Game_Of_Thrones +
                                                 book2_A_Clash_Of_Kings +
                                                 book3_A_Storm_Of_Swords +
                                                 book4_A_Feast_For_Crows +
                                                 book5_A_Dance_with_Dragons +
                                                 number_of_names +
                                                 dateOfBirth +
                                                 age +
                                                 numDeadRelations +
                                                 popularity""",
                           data    = got_df_train)


# fitting the model object
results_logistic = logistic_full.fit()


# checking the results summary
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.421981
         Iterations 28


<class 'statsmodels.iolib.summary2.Summary'>
"""
                                         Results: Logit
================================================================================================
Model:                         Logit                      Pseudo R-squared:           0.256     
Dependent Variable:            isAlive                    AIC:                        1535.7790 
Date:                          2022-01-14 14:47           BIC:                        1694.3493 
No. Observations:              1751                       Log-Likelihood:             -738.89   
Df Model:                      28                         LL-Null:                    -992.53   
Df Residuals:                  1722                       LLR p-value:                2.1232e-89
Converged:                     1.0000                     Scale:                      1.0000    
No. Iterations:                28.0000                                                          
------------------------------------------------------------------------------------------------
                            Coef.      Std.Err.       z    P>|z|       [0.025         0.975]    
------------------------------------------------------------------------------------------------
Intercept                  -62.6865         8.2967 -7.5556 0.0000        -78.9477       -46.4253
title_unknown                0.1261         0.4646  0.2713 0.7861         -0.7846         1.0367
title_ser                   -0.4272         0.2106 -2.0287 0.0425         -0.8398        -0.0145
title_dozen                  0.3477         0.3653  0.9519 0.3411         -0.3682         1.0636
culture_unknown              0.3887         0.2263  1.7178 0.0858         -0.0548         0.8322
culture_hundred              0.3415         0.2873  1.1887 0.2346         -0.2216         0.9046
culture_dozen                0.1651         0.2734  0.6040 0.5459         -0.3708         0.7011
house_unknown                0.0619         0.1957  0.3165 0.7516         -0.3216         0.4455
house_nwatch                -1.2506         0.2686 -4.6561 0.0000         -1.7770        -0.7242
house_dozen                 -0.3924         0.1704 -2.3037 0.0212         -0.7263        -0.0586
mother_alive                -0.7185         1.3188 -0.5448 0.5859         -3.3034         1.8663
mother_dead                 -1.2373         1.7775 -0.6961 0.4864         -4.7211         2.2466
father_alive                -0.5292         1.4646 -0.3613 0.7179         -3.3998         2.3415
father_dead                 -0.3731         1.1343 -0.3290 0.7422         -2.5963         1.8501
heir_alive                   0.3889         1.1482  0.3387 0.7349         -1.8616         2.6394
heir_dead                  -40.5751 438602043.6799 -0.0000 1.0000 -859644249.7333 859644168.5832
spouse_alive                 0.2408         0.2456  0.9806 0.3268         -0.2405         0.7222
spouse_dead                 -0.1059         0.4489 -0.2358 0.8136         -0.9857         0.7740
isNoble                      0.2166         0.4790  0.4521 0.6512         -0.7223         1.1555
book1_A_Game_Of_Thrones     -0.5780         0.1717 -3.3670 0.0008         -0.9145        -0.2415
book2_A_Clash_Of_Kings      -0.7582         0.1558 -4.8662 0.0000         -1.0635        -0.4528
book3_A_Storm_Of_Swords     -0.3915         0.1635 -2.3941 0.0167         -0.7120        -0.0710
book4_A_Feast_For_Crows      1.3625         0.1682  8.1015 0.0000          1.0329         1.6921
book5_A_Dance_with_Dragons  -0.1032         0.1605 -0.6432 0.5201         -0.4177         0.2113
number_of_names             -0.1158         0.0983 -1.1774 0.2390         -0.3085         0.0769
dateOfBirth                  0.2165         0.0281  7.6946 0.0000          0.1614         0.2717
age                          0.2166         0.0281  7.6955 0.0000          0.1614         0.2717
numDeadRelations            -0.0485         0.0701 -0.6912 0.4894         -0.1858         0.0889
popularity     

Remove insignificant values and improve the score

In [25]:
# instantiating a logistic regression model object
logistic_sig1 = smf.logit(formula = """isAlive ~ title_ser +
                                                 house_nwatch +
                                                 house_dozen +
                                                 book1_A_Game_Of_Thrones +
                                                 book2_A_Clash_Of_Kings +
                                                 book3_A_Storm_Of_Swords +
                                                 book4_A_Feast_For_Crows +
                                                 dateOfBirth +
                                                 age +
                                                 popularity""",
                                 data    = got_df_train)


# fitting the model object
results_logistic = logistic_sig1.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.427663
         Iterations 11


<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Results: Logit
==========================================================================
Model:                 Logit               Pseudo R-squared:    0.246     
Dependent Variable:    isAlive             AIC:                 1519.6745 
Date:                  2022-01-14 14:47    BIC:                 1579.8219 
No. Observations:      1751                Log-Likelihood:      -748.84   
Df Model:              10                  LL-Null:             -992.53   
Df Residuals:          1740                LLR p-value:         2.1786e-98
Converged:             1.0000              Scale:               1.0000    
No. Iterations:        11.0000                                            
--------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|   [0.025   0.975] 
--------------------------------------------------------------------------
Intercept               -64.7509   8.3546 -7.7503 0.0000 -81.1256 -48.3762
title_ser                -0.3440   0.1577 -2.1808 0.0292  -0.6531  -0.0348
house_nwatch             -1.2304   0.2498 -4.9255 0.0000  -1.7200  -0.7408
house_dozen              -0.3598   0.1534 -2.3453 0.0190  -0.6605  -0.0591
book1_A_Game_Of_Thrones  -0.5615   0.1640 -3.4246 0.0006  -0.8829  -0.2402
book2_A_Clash_Of_Kings   -0.6608   0.1478 -4.4719 0.0000  -0.9504  -0.3712
book3_A_Storm_Of_Swords  -0.3539   0.1574 -2.2486 0.0245  -0.6623  -0.0454
book4_A_Feast_For_Crows   1.3812   0.1536  8.9945 0.0000   1.0802   1.6821
dateOfBirth               0.2243   0.0284  7.9015 0.0000   0.1686   0.2799
age                       0.2243   0.0284  7.9025 0.0000   0.1687   0.2799
popularity               -2.8716   0.3984 -7.2076 0.0000  -3.6525  -2.0907
==========================================================================

"""

Further remove more variables with comparatively higher p - value.

In [26]:
# instantiating a logistic regression model object
logistic_sig2 = smf.logit(formula = """isAlive ~ house_nwatch +
                                                 book1_A_Game_Of_Thrones +
                                                 book2_A_Clash_Of_Kings +
                                                 book4_A_Feast_For_Crows +
                                                 dateOfBirth +
                                                 age +
                                                 popularity""",
                                 data    = got_df_train)


# fitting the model object
results_logistic = logistic_sig2.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.431953
         Iterations 10


<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Results: Logit
==========================================================================
Model:                 Logit               Pseudo R-squared:    0.238     
Dependent Variable:    isAlive             AIC:                 1528.6991 
Date:                  2022-01-14 14:47    BIC:                 1572.4426 
No. Observations:      1751                Log-Likelihood:      -756.35   
Df Model:              7                   LL-Null:             -992.53   
Df Residuals:          1743                LLR p-value:         6.9589e-98
Converged:             1.0000              Scale:               1.0000    
No. Iterations:        10.0000                                            
--------------------------------------------------------------------------
                         Coef.   Std.Err.    z    P>|z|   [0.025   0.975] 
--------------------------------------------------------------------------
Intercept               -61.9856   8.1652 -7.5914 0.0000 -77.9892 -45.9820
house_nwatch             -1.1955   0.2372 -5.0408 0.0000  -1.6604  -0.7307
book1_A_Game_Of_Thrones  -0.5662   0.1632 -3.4700 0.0005  -0.8860  -0.2464
book2_A_Clash_Of_Kings   -0.7432   0.1425 -5.2160 0.0000  -1.0225  -0.4640
book4_A_Feast_For_Crows   1.1935   0.1364  8.7513 0.0000   0.9262   1.4608
dateOfBirth               0.2142   0.0277  7.7279 0.0000   0.1599   0.2686
age                       0.2143   0.0277  7.7289 0.0000   0.1599   0.2686
popularity               -2.9388   0.4030 -7.2917 0.0000  -3.7287  -2.1489
==========================================================================

"""

Generate dictionary of explanatory variables

In [27]:
candidate_dict = {

 # full model
 'full'   : [ 'title_unknown' , 'title_ser' , 'title_dozen' , 'culture_unknown' , \
                   'culture_hundred' , 'culture_dozen' , 'house_unknown' , 'house_nwatch' , \
                   'house_dozen' , 'mother_alive' , 'mother_dead' , 'father_alive' , \
                   'father_dead' , 'heir_alive' , 'heir_dead' , 'spouse_alive', \
                   'spouse_dead' , 'isNoble' , 'book1_A_Game_Of_Thrones' , \
                   'book2_A_Clash_Of_Kings' , 'book3_A_Storm_Of_Swords' , \
                   'book4_A_Feast_For_Crows' , 'book5_A_Dance_with_Dragons' , \
                   'number_of_names','dateOfBirth','age','numDeadRelations' , \
                   'popularity' ] ,
 

 # significant variables only (set 1)
 'sig_1'    : [ 'title_ser' , 'house_nwatch' , 'house_dozen' , 'book1_A_Game_Of_Thrones' , \
                   'book2_A_Clash_Of_Kings' , 'book3_A_Storm_Of_Swords' ,
                   'book4_A_Feast_For_Crows' , 'dateOfBirth','age' , 'popularity' ],
    
    
 # significant variables only (set 2)
 'sig_2'  : [ 'house_nwatch','book1_A_Game_Of_Thrones' , 'book2_A_Clash_Of_Kings' ,\
                   'book4_A_Feast_For_Crows' , 'dateOfBirth' , 'age' , 'popularity' ],

}

Setting <strong>sig_1</strong> as independents that will be used.

Run a logistic regression as default.

In [28]:
# train/test split with the sig_1
got_df_data   =  got_df.loc[ : , candidate_dict['sig_1']]
got_df_target =  got_df.loc[ : , 'isAlive']

# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            got_df_data,
            got_df_target,
            random_state = 219,
            test_size    = 0.1,
            stratify     = got_df_target)


# instantiate a logistic regression model
lr_default = LogisticRegression(solver = 'lbfgs',
                                     C = 1,
                              max_iter = 10000,
                          random_state = 219)


# fitting the training data
lr_default_fit = lr_default.fit(x_train, y_train)


# predicting based on the testing set
lr_default_fit_pred = lr_default_fit.predict(x_test)


# scoring the results
print('LogReg Training ACCURACY:', lr_default_fit.score(x_train, y_train).round(4))
print('LogReg Testing  ACCURACY:', lr_default_fit.score(x_test, y_test).round(4))

# saving scoring data for future use
lr_default_train_score = lr_default_fit.score(x_train, y_train).round(4) # accuracy
lr_default_test_score  = lr_default_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('LogReg Train-Test Gap   :', abs(lr_default_train_score - lr_default_test_score).round(4))
lr_default_test_gap = abs(lr_default_train_score - lr_default_test_score).round(4)

# scoring with AUC
print('AUC Score               :', roc_auc_score(y_true  = y_test,
                                          y_score = lr_default_fit_pred).round(4))


# saving AUC score
lr_default_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = lr_default_fit_pred).round(4)

LogReg Training ACCURACY: 0.7761
LogReg Testing  ACCURACY: 0.8154
LogReg Train-Test Gap   : 0.0393
AUC Score               : 0.6728


<h4>Generate confusion matrix and save.</h4>

In [29]:
# generate consusion matrix
lr_default_tn, \
lr_default_fp, \
lr_default_fn, \
lr_default_tp = confusion_matrix(y_true = y_test, y_pred = lr_default_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {lr_default_tn}
False Positives: {lr_default_fp}
False Negatives: {lr_default_fn}
True Positives : {lr_default_tp}
""")


True Negatives : 19
False Positives: 31
False Negatives: 5
True Positives : 140



<h4>Hyperparameter Tuning.</h4>

In [30]:
# logistic Regression's hyperparameter tuning

# declaring a hyperparameter space
C_range          = np.arange(1.0, 3.0, 1.0)
warm_start_range = [True, False]
solver_range     = ['newton-cg', 'sag', 'lbfgs']


# creating a hyperparameter grid
param_grid = {'C'          : C_range,
              'warm_start' : warm_start_range,
              'solver'     : solver_range}


# instantiating the model object without hyperparameters
lr_tuned = LogisticRegression(random_state = 219,
                              max_iter     = 100000000) # increased for convergence


# GridSearchCV object
lr_tuned_cv = RandomizedSearchCV(estimator           = lr_tuned,   # the model object
                                 param_distributions = param_grid, # parameters to tune
                                 cv                  = 3,          # how many folds in cross-validation
                                 n_iter              = 15,        # number of combinations of hyperparameters to try
                                 random_state        = 219,        # starting point for random sequence
                                 scoring = make_scorer(
                                           roc_auc_score,
                                           needs_threshold = False)) # scoring criteria (AUC)


# fitting to the FULL DATASET (due to cross-validation)
lr_tuned_cv.fit(got_df_data, got_df_target)


# predict step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", lr_tuned_cv.best_params_)
print("Tuned CV AUC      :", lr_tuned_cv.best_score_.round(4))

C:\Users\Sneha R\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 12 is smaller than n_iter=15. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Sneha R\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Sneha R\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Sneha R\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Sneha R\AppData

Tuned Parameters  : {'warm_start': True, 'solver': 'newton-cg', 'C': 2.0}
Tuned CV AUC      : 0.6607


<h4>Reset Logistic Regression.</h4>

In [31]:
# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            got_df_data,
            got_df_target,
            random_state = 219,
            test_size    = 0.1,
            stratify     = got_df_target)


# instantiating a logistic regression model
lr_tuned = LogisticRegression(solver = 'newton-cg',
                                     C = 2.0,
                              max_iter = 10000,
                            warm_start = True,
                          random_state = 219)

# fitting the training data
lr_tuned_fit = lr_tuned.fit(got_df_data, got_df_target)


# predicting based on the testing set
lr_tuned_fit_pred = lr_tuned_fit.predict(x_test)

# scoring the results
print('LogReg Training ACCURACY:', lr_tuned_fit.score(x_train, y_train).round(4))
print('LogReg Testing  ACCURACY:', lr_tuned_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
lr_tuned_train_score = lr_tuned_fit.score(x_train, y_train).round(4) # accuracy
lr_tuned_test_score  = lr_tuned_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('LogReg Train-Test Gap   :', abs(lr_tuned_train_score - lr_tuned_test_score).round(4))
lr_tuned_test_gap = abs(lr_tuned_train_score - lr_tuned_test_score).round(4)

# scoring with AUC
print('AUC Score               :', roc_auc_score(y_true  = y_test,
                                          y_score = lr_tuned_fit_pred).round(4))


# saving AUC score
lr_tuned_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = lr_tuned_fit_pred).round(4)

LogReg Training ACCURACY: 0.8115
LogReg Testing  ACCURACY: 0.8256
LogReg Train-Test Gap   : 0.0141
AUC Score               : 0.6993


<h4>Generate confusion matrix and save.</h4>

In [32]:
# generate consusion matrix
lr_tuned_tn, \
lr_tuned_fp, \
lr_tuned_fn, \
lr_tuned_tp = confusion_matrix(y_true = y_test, y_pred = lr_tuned_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {lr_tuned_tn}
False Positives: {lr_tuned_fp}
False Negatives: {lr_tuned_fn}
True Positives : {lr_tuned_tp}
""")


True Negatives : 22
False Positives: 28
False Negatives: 6
True Positives : 139



<h3>Classification Tree</h3>

<h4>Generate a default model.

In [33]:
# instantiate classification tree
tree_default = DecisionTreeClassifier(max_depth = 8,
                                     min_samples_leaf = 25,
                                     random_state = 219)


# fitting the training data
tree_default_fit  = tree_default.fit(x_train, y_train)


# predicting on new data
tree_default_fit_pred = tree_default_fit.predict(x_test)


# scoring the results
print('CART Training ACCURACY:', tree_default_fit.score(x_train, y_train).round(4))
print('CART Testing  ACCURACY:', tree_default_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
tree_default_train_score = tree_default_fit.score(x_train, y_train).round(4) # accuracy
tree_default_test_score  = tree_default_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('CART Train-Test Gap   :', abs(tree_default_train_score - tree_default_test_score).round(4))
tree_default_test_gap = abs(tree_default_train_score - tree_default_test_score).round(4)

# scoring with AUC
print('AUC Score             :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_default_fit_pred).round(4))


# saving AUC score
tree_default_auc_score = roc_auc_score(y_true  = y_test,
                                       y_score = tree_default_fit_pred).round(4)

CART Training ACCURACY: 0.8247
CART Testing  ACCURACY: 0.8718
CART Train-Test Gap   : 0.0471
AUC Score             : 0.8024


<h4>Generate confusion matrix and save.</h4>

In [34]:
# generate confusion matrix
tree_default_tn, \
tree_default_fp, \
tree_default_fn, \
tree_default_tp = confusion_matrix(y_true = y_test, y_pred = tree_default_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tree_default_tn}
False Positives: {tree_default_fp}
False Negatives: {tree_default_fn}
True Positives : {tree_default_tp}
""")


True Negatives : 33
False Positives: 17
False Negatives: 8
True Positives : 137



<h4>Hyperparameter Tuning.</h4>

In [35]:
# declaring a hyperparameter space
criterion_range = ['gini', 'entropy']
splitter_range  = ['best', 'random']
leaf_range      = np.arange(1, 3, 1)
depth_range     = np.arange(7, 9, 1)


# creating a hyperparameter grid
param_grid = {'criterion'        : criterion_range,
              'splitter'         : splitter_range,
              'max_depth'        : depth_range,
              'min_samples_leaf' : leaf_range}


# instantiate the model object without hyperparameters
tuned_tree = DecisionTreeClassifier(random_state = 219)


# RandomizedSearchCV object
tuned_tree_cv = RandomizedSearchCV(estimator             = tuned_tree,
                                   param_distributions   = param_grid,
                                   cv                    = 3,
                                   n_iter                = 16,
                                   random_state          = 219,
                                   scoring = make_scorer(roc_auc_score,
                                             needs_threshold = False))


# fitting to the full data set (due to cross-validation)
tuned_tree_cv.fit(got_df_data, got_df_target)


# predict step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", tuned_tree_cv.best_params_)
print("Tuned Training AUC:", tuned_tree_cv.best_score_.round(4))

Tuned Parameters  : {'splitter': 'best', 'min_samples_leaf': 1, 'max_depth': 8, 'criterion': 'entropy'}
Tuned Training AUC: 0.6957


<h4>Reset Classification Tree model.</h4>

In [36]:
# building a model based on hyperparameter tuning results

# instantiate a logistic regression model with tuned values
tree_tuned = DecisionTreeClassifier(splitter         = 'best',
                                    min_samples_leaf = 1,
                                    max_depth        = 8,
                                    criterion        = 'entropy',
                                    random_state     = 219)


# fitting to the FULL DATASET (due to cross-validation)
tree_tuned_fit = tree_tuned.fit(got_df_data, got_df_target)


# predicting based on the testing set
tree_tuned_fit_pred = tree_tuned.predict(x_test)


# scoring the results
print('CART_tuned Training ACCURACY:', tree_tuned_fit.score(x_train, y_train).round(4))
print('CART_tuned Testing  ACCURACY:', tree_tuned_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
tree_tuned_train_score = tree_tuned_fit.score(x_train, y_train).round(4) # accuracy
tree_tuned_test_score  = tree_tuned_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('CART_tuned Train-Test Gap   :', abs(tree_tuned_train_score - tree_tuned_test_score).round(4))
tree_tuned_test_gap = abs(tree_tuned_train_score - tree_tuned_test_score).round(4)

# scoring with AUC
print('AUC Score                   :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_tuned_fit_pred).round(4))


# saving AUC score
tree_tuned_auc_score = roc_auc_score(y_true  = y_test,
                                     y_score = tree_tuned_fit_pred).round(4)

CART_tuned Training ACCURACY: 0.8527
CART_tuned Testing  ACCURACY: 0.8872
CART_tuned Train-Test Gap   : 0.0345
AUC Score                   : 0.8128


<h4>Generate confusion matrix and save.</h4>

In [37]:
# generate confusion matrix
tree_tuned_tn, \
tree_tuned_fp, \
tree_tuned_fn, \
tree_tuned_tp = confusion_matrix(y_true = y_test, y_pred = tree_tuned_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {tree_tuned_tn}
False Positives: {tree_tuned_fp}
False Negatives: {tree_tuned_fn}
True Positives : {tree_tuned_tp}
""")


True Negatives : 33
False Positives: 17
False Negatives: 5
True Positives : 140



<h3>Random Forest</h3>

<h4>Generate a default model.</h4>

In [38]:
# random forest model
rf_default = RandomForestClassifier(    n_estimators   = 600,
                                    criterion        = 'entropy',
                                    max_depth        = 8,
                                    min_samples_leaf = 1,
                                    bootstrap        = False,
                                    warm_start       = True,
                                    random_state     = 219)


# fitting the training data
rf_default_fit  = rf_default.fit(x_train, y_train)


# predicting on new data
rf_default_fit_pred = tree_default_fit.predict(x_test)


# scoring the results
print('rf Training ACCURACY:', rf_default_fit.score(x_train, y_train).round(4))
print('rf Testing  ACCURACY:', rf_default_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
rf_default_train_score = rf_default_fit.score(x_train, y_train).round(4) # accuracy
rf_default_test_score  = rf_default_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('rf Train-Test Gap   :', abs(rf_default_train_score - rf_default_test_score).round(4))
rf_default_test_gap = abs(rf_default_train_score - rf_default_test_score).round(4)


# scoring with AUC
print('AUC Score           :', roc_auc_score(y_true  = y_test,
                                             y_score = rf_default_fit_pred).round(4))


# saving AUC score
rf_default_auc_score = roc_auc_score(y_true  = y_test,
                                     y_score = rf_default_fit_pred).round(4)

rf Training ACCURACY: 0.8635
rf Testing  ACCURACY: 0.8769
rf Train-Test Gap   : 0.0134
AUC Score           : 0.8024


<h4>Generate confusion matrix and save.</h4>

In [39]:
# generate confusion matrix
rf_default_tn, \
rf_default_fp, \
rf_default_fn, \
rf_default_tp = confusion_matrix(y_true = y_test, y_pred = rf_default_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {rf_default_tn}
False Positives: {rf_default_fp}
False Negatives: {rf_default_fn}
True Positives : {rf_default_tp}
""")


True Negatives : 33
False Positives: 17
False Negatives: 8
True Positives : 137



<h4>Hyperparameter Tuning.</h4>

In [40]:
# declaring a hyperparameter space
estimator_range  = np.arange(100, 300, 100)
leaf_range       = np.arange(1, 3, 1)
depth_range      = np.arange(6 ,8, 1)
criterion_range  = ['gini', 'entropy']
bootstrap_range  = [True, False]
warm_start_range = [True, False]


# creating a hyperparameter grid
param_grid = {'n_estimators'     : estimator_range,
              'min_samples_leaf' : leaf_range,
              'criterion'        : criterion_range,
              'bootstrap'        : bootstrap_range,
              'max_depth'        : depth_range,
              'warm_start'       : warm_start_range}


# instantiate the model object without hyperparameters
forest_grid = RandomForestClassifier(random_state = 219,
                                    max_depth     = 8)


# GridSearchCV object
forest_cv = RandomizedSearchCV(estimator           = forest_grid,
                               param_distributions = param_grid,
                               cv                  = 3,
                               n_iter              = 64,
                               scoring             = make_scorer(roc_auc_score,
                                            needs_threshold = False))


# fitting to the FULL DATASET (due to cross-validation)
forest_cv.fit(got_df_data, got_df_target)


# predict step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", forest_cv.best_params_)
print("Tuned Training AUC:", forest_cv.best_score_.round(4))

Tuned Parameters  : {'warm_start': True, 'n_estimators': 100, 'min_samples_leaf': 1, 'max_depth': 7, 'criterion': 'gini', 'bootstrap': False}
Tuned Training AUC: 0.6737


<h4>Reset Random Forest model.</h4>

In [41]:
# instantiate the model object with hyperparameters
rf_tuned = RandomForestClassifier(n_estimators   = 100,
                                    criterion        = 'gini',
                                    max_depth        = 7,
                                    min_samples_leaf = 1,
                                    bootstrap        = False,
                                    warm_start       = True,
                                    random_state     = 219)


# fitting to the full data set (due to cross-validation)
rf_tuned_fit = rf_tuned.fit(got_df_data, got_df_target)


# predicting based on the testing set
rf_tuned_fit_pred = rf_tuned_fit.predict(x_test)


# soring the results
print('rf_tuned Training ACCURACY:', rf_tuned_fit.score(x_train, y_train).round(4))
print('rf_tuned Testing  ACCURACY:', rf_tuned_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
rf_tuned_train_score = rf_tuned_fit.score(x_train, y_train).round(4) # accuracy
rf_tuned_test_score  = rf_tuned_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('rf_tuned Train-Test Gap   :', abs(rf_tuned_train_score - rf_tuned_test_score).round(4))
rf_tuned_test_gap = abs(rf_tuned_train_score - rf_tuned_test_score).round(4)

# sscoring with AUC
print('AUC Score                 :', roc_auc_score(y_true  = y_test,
                                          y_score = rf_tuned_fit_pred).round(4))


# saving AUC score
rf_tuned_auc_score = roc_auc_score(y_true  = y_test,
                                       y_score = rf_tuned_fit_pred).round(4)

rf_tuned Training ACCURACY: 0.8441
rf_tuned Testing  ACCURACY: 0.9026
rf_tuned Train-Test Gap   : 0.0585
AUC Score                 : 0.8166


<h4>Generate confusion matrix and save.</h4>

In [42]:
# generate confusion matrix and save
rf_tuned_tn, \
rf_tuned_fp, \
rf_tuned_fn, \
rf_tuned_tp = confusion_matrix(y_true = y_test, y_pred = rf_tuned_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {rf_tuned_tn}
False Positives: {rf_tuned_fp}
False Negatives: {rf_tuned_fn}
True Positives : {rf_tuned_tp}
""")


True Negatives : 32
False Positives: 18
False Negatives: 1
True Positives : 144



<h3>Gradient Boosted Models (GBM)</h3>

<h4>Generate a default model.</h4>

In [43]:
# instantiate the model object without hyperparameters
gbm_default = GradientBoostingClassifier(learning_rate = 0.1,
                                         max_depth     = 5,
                                         n_estimators  = 150,
                                         warm_start    = True,
                                         random_state  = 219)


# fitting step is needed as we are not using .best_estimator
gbm_default_fit = gbm_default.fit(x_train, y_train)


# predicting based on the testing set
gbm_default_fit_pred = gbm_default_fit.predict(x_test)


# scoring the results
print('GBM Training ACCURACY:', gbm_default_fit.score(x_train, y_train).round(4))
print('GBM Testing  ACCURACY:', gbm_default_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
gbm_default_train_score = gbm_default_fit.score(x_train, y_train).round(4) # accuracy
gbm_default_test_score  = rf_default_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('GBM Train-Test Gap   :', abs(gbm_default_train_score - gbm_default_test_score).round(4))
gbm_default_test_gap = abs(gbm_default_train_score - gbm_default_test_score).round(4)


# scoring with AUC
print('AUC Score            :', roc_auc_score(y_true  = y_test,
                                              y_score = gbm_default_fit_pred).round(4))


# saving AUC score
gbm_default_auc_score = roc_auc_score(y_true  = y_test,
                                      y_score = gbm_default_fit_pred).round(4)

GBM Training ACCURACY: 0.9121
GBM Testing  ACCURACY: 0.8615
GBM Train-Test Gap   : 0.0352
AUC Score            : 0.789


<h4>Generate confusion matrix and save.</h4>

In [44]:
# generate confusion matrix
gbm_default_tn, \
gbm_default_fp, \
gbm_default_fn, \
gbm_default_tp = confusion_matrix(y_true = y_test, y_pred = gbm_default_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_default_tn}
False Positives: {gbm_default_fp}
False Negatives: {gbm_default_fn}
True Positives : {gbm_default_tp}
""")


True Negatives : 32
False Positives: 18
False Negatives: 9
True Positives : 136



<h4>Hyperparameter Tuning.</h4>

In [45]:
# declaring a hyperparameter space
learn_range        = np.arange(0.3, 0.5, 0.1)
estimator_range    = np.arange(160, 200, 20)
depth_range        = np.arange(2, 6, 2)
warm_start_range   = [True, False]


# creating a hyperparameter grid
param_grid = {'learning_rate' : learn_range,
              'max_depth'     : depth_range,
              'n_estimators'  : estimator_range,
              'warm_start'    : warm_start_range}


# instantiate the model object without hyperparameters
gbm_grid = GradientBoostingClassifier(random_state = 219)


# GridSearchCV object
gbm_cv = RandomizedSearchCV(estimator          = gbm_grid,
                           param_distributions = param_grid,
                           cv                  = 3,
                           n_iter              = 16,
                           random_state        = 219,
                           scoring             = make_scorer(roc_auc_score,
                                                 needs_threshold = False))


# fitting to the full dataset (due to cross-validation)
gbm_cv.fit(got_df_data, got_df_target)


# predict step is not needed


# printing the optimal parameters and best score
print("Tuned Parameters  :", gbm_cv.best_params_)
print("Tuned Training AUC:", gbm_cv.best_score_.round(4))

Tuned Parameters  : {'warm_start': True, 'n_estimators': 180, 'max_depth': 4, 'learning_rate': 0.4}
Tuned Training AUC: 0.7092


<h4>Reset Gradient Boosting model.</h4>

In [46]:
# instantiate with best_estimator
gbm_tuned = GradientBoostingClassifier(learning_rate = 0.4,
                                       max_depth     = 4,
                                       n_estimators  = 180,
                                       warm_start    = True,
                                       random_state  = 219)


# fitting to the FULL DATASET (due to cross-validation)
gbm_tuned_fit = gbm_tuned.fit(got_df_data, got_df_target)


# predicting based on the testing set
gbm_tuned_fit_pred = gbm_tuned_fit.predict(x_test)


# scoring the results
print('GBM_tuned Training ACCURACY:', gbm_tuned_fit.score(x_train, y_train).round(4))
print('GBM_tuned Testing  ACCURACY:', gbm_tuned_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
gbm_tuned_train_score = gbm_tuned_fit.score(x_train, y_train).round(4) # accuracy
gbm_tuned_test_score  = gbm_tuned_fit.score(x_test, y_test).round(4)   # accuracy


# displaying and saving the gap between training and testing
print('GBM_tuned Train-Test Gap   :', abs(gbm_tuned_train_score - gbm_tuned_test_score).round(4))
gbm_tuned_test_gap = abs(gbm_tuned_train_score - gbm_tuned_test_score).round(4)

# SCORING with AUC
print('AUC Score                  :', roc_auc_score(y_true  = y_test,
                                          y_score = gbm_tuned_fit_pred).round(4))


# saving AUC score
gbm_tuned_auc_score = roc_auc_score(y_true  = y_test,
                                       y_score = gbm_tuned_fit_pred).round(4)

GBM_tuned Training ACCURACY: 0.9298
GBM_tuned Testing  ACCURACY: 0.9538
GBM_tuned Train-Test Gap   : 0.024
AUC Score                  : 0.9231


<h4>Generate confusion matrix and save.</h4>

In [47]:
# generate confusion matrix
gbm_tuned_tn, \
gbm_tuned_fp, \
gbm_tuned_fn, \
gbm_tuned_tp = confusion_matrix(y_true = y_test, y_pred = gbm_tuned_fit_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {gbm_tuned_tn}
False Positives: {gbm_tuned_fp}
False Negatives: {gbm_tuned_fn}
True Positives : {gbm_tuned_tp}
""")


True Negatives : 43
False Positives: 7
False Negatives: 2
True Positives : 143



<h2>Summary</h2>

In [54]:
# summary table
print(f"""
Model          AUC Score      TN, FP, FN, TP
-----          ---------      --------------
Logistic       {lr_default_auc_score}         {lr_default_tn, lr_default_fp, lr_default_fn, lr_default_tp}
Logistic_tuned {lr_tuned_auc_score}         {lr_tuned_tn, lr_tuned_fp, lr_tuned_fn, lr_tuned_tp}
CART           {tree_default_auc_score}         {tree_default_tn, tree_default_fp, tree_default_fn, tree_default_tp}
CART_tuned     {tree_tuned_auc_score}         {tree_tuned_tn, tree_tuned_fp, tree_tuned_fn, tree_tuned_tp}
RandomForest   {rf_default_auc_score}         {rf_default_tn, rf_default_fp, rf_default_fn, rf_default_tp}
Forest_tuned   {rf_tuned_auc_score}         {rf_tuned_tn, rf_tuned_fp, rf_tuned_fn, rf_tuned_tp}
GBM            {gbm_default_auc_score}          {gbm_default_tn, gbm_default_fp, gbm_default_fn, gbm_default_tp}
GBM_tuned      {gbm_tuned_auc_score}         {gbm_tuned_tn, gbm_tuned_fp, gbm_tuned_fn, gbm_tuned_tp}
""")


# creating a dictionary for model results
model_performance = {
    
    'Model Name'    : ['Logistic', 'Logistic_tuned', 'CART', 'CART_tuned',
                       'RandomForest', 'Forest_tuned', 'GBM', 'GBM_tuned'],
           
    'AUC Score' : [lr_default_auc_score, lr_tuned_auc_score,
                   tree_default_auc_score, tree_tuned_auc_score,
                   rf_default_auc_score, rf_tuned_auc_score,
                   gbm_default_auc_score, gbm_tuned_auc_score],
    
    'Training Accuracy' : [lr_default_train_score, lr_tuned_train_score,
                           tree_default_train_score, tree_tuned_train_score,
                           rf_default_train_score, rf_tuned_train_score,
                           gbm_default_train_score, gbm_tuned_train_score],
           
    'Testing Accuracy'  : [lr_default_test_score, lr_tuned_test_score,
                           tree_default_test_score, tree_tuned_test_score,
                           rf_default_test_score, rf_tuned_test_score,
                           gbm_default_test_score, gbm_tuned_test_score],

    'Confusion Matrix'  : [(lr_default_tn, lr_default_fp, lr_default_fn, lr_default_tp),
                           (lr_tuned_tn, lr_tuned_fp, lr_tuned_fn, lr_tuned_tp),
                           (tree_default_tn, tree_default_fp, tree_default_fn, tree_default_tp),
                           (tree_tuned_tn, tree_tuned_fp, tree_tuned_fn, tree_tuned_tp),
                           (rf_default_tn, rf_default_fp, rf_default_fn, rf_default_tp),
                           (rf_tuned_tn, rf_tuned_fp, rf_tuned_fn, rf_tuned_tp),
                           (gbm_default_tn, gbm_default_fp, gbm_default_fn, gbm_default_tp),
                           (gbm_tuned_tn, gbm_tuned_fp , gbm_tuned_fn, gbm_tuned_tp)]}


# converting model_performance into a DataFrame
model_performance = pd.DataFrame(model_performance)
max_auc = model_performance['AUC Score'].max()

print(model_performance)

#output decision
print(f"""
\n\n
Therefore the ideal choice is\n
{model_performance.loc[:,["Model Name"]][model_performance.loc[:, "AUC Score"] == max_auc]}
\n
Its AUC value is 
{float(model_performance["AUC Score"][model_performance.loc[: , "AUC Score"] == max_auc])}
\n
Its train test gap is 
{abs(float(model_performance["Training Accuracy"][model_performance.loc[: , "AUC Score"] == max_auc]) -
float(model_performance["Testing Accuracy"][model_performance.loc[: , "AUC Score"] == max_auc]))},
acceptable.
\n\n

""")


Model          AUC Score      TN, FP, FN, TP
-----          ---------      --------------
Logistic       0.6728         (19, 31, 5, 140)
Logistic_tuned 0.6993         (22, 28, 6, 139)
CART           0.8024         (33, 17, 8, 137)
CART_tuned     0.8128         (33, 17, 5, 140)
RandomForest   0.8024         (33, 17, 8, 137)
Forest_tuned   0.8166         (32, 18, 1, 144)
GBM            0.789          (32, 18, 9, 136)
GBM_tuned      0.9231         (43, 7, 2, 143)

       Model Name  AUC Score  Training Accuracy  Testing Accuracy  Confusion Matrix
0        Logistic     0.6728             0.7761            0.8154  (19, 31, 5, 140)
1  Logistic_tuned     0.6993             0.8115            0.8256  (22, 28, 6, 139)
2            CART     0.8024             0.8247            0.8718  (33, 17, 8, 137)
3      CART_tuned     0.8128             0.8527            0.8872  (33, 17, 5, 140)
4    RandomForest     0.8024             0.8635            0.8769  (33, 17, 8, 137)
5    Forest_tuned     0.8166 